In [4]:
!pip install spacy==3.1.4
!pip install matplotlib
!pip install scikit-learn

^C
Defaulting to user installation because normal site-packages is not writeable

  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [651 lines of output]
    Using cached setuptools-75.2.0-py3-none-any.whl.metadata (6.9 kB)
    Using cached Cython-0.29.37-py2.py3-none-any.whl.metadata (3.1 kB)
    Using cached cymem-2.0.8-cp312-cp312-win_amd64.whl.metadata (8.6 kB)
    Using cached preshed-3.0.9-cp312-cp312-win_amd64.whl.metadata (2.2 kB)
    Using cached murmurhash-1.0.10-cp312-cp312-win_amd64.whl.metadata (2.0 kB)
    Using cached thinc-8.0.17.tar.gz (189 kB)
    Installing build dependencies: started
    Installing build dependencies: still running...
    Installing build dependencies: finished with status 'done'
    Getting requirements to build wheel: started
    Getting requirements to build wheel: finished with status 'done'
    Preparing metadata (pyproject.toml): started
    Preparing metadata (pyproject.toml): finished with status 'done'
    Using cached blis-0.7.11-cp312-c


  Using cached spacy-3.1.4.tar.gz (1.0 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import spacy
import random
import json
from spacy.training import Example
from spacy.util import minibatch, compounding
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
import os

In [3]:
# Download the large Portuguese model and lookups data if not already present
if not spacy.util.is_package('pt_core_news_lg'):
    os.system("python -m spacy download pt_core_news_lg")
    print("Downloaded pt_core_news_lg model.")

if not spacy.util.is_package('spacy-lookups-data'):
    os.system("python -m spacy download spacy-lookups-data")
    print("Downloaded spacy-lookups-data package.")


Downloaded pt_core_news_lg model.
Downloaded spacy-lookups-data package.


In [81]:
# Load training data from JSON file
def load_training_data(file_path):
    print(f"Loading training data from {file_path}")
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [82]:
# Remove overlapping or duplicate entities
def remove_overlapping_entities(entities):
    print(f"Removing overlapping entities from {entities}")
    unique_entities = []
    seen = set()
    for start, end, label in entities:
        if (start, end) not in seen:
            seen.add((start, end))
            unique_entities.append((start, end, label))
    return unique_entities

In [111]:
from spacy.training import Example

# Função aprimorada de treinamento do SpaCy
def train_spacy_model(training_data, output_dir, n_iter=40):
    try:
        print("Loading pre-trained SpaCy model 'pt_core_news_lg'")
        # Carregar o modelo pré-treinado do SpaCy
        nlp = spacy.load('pt_core_news_lg')
        print("Model loaded successfully.")

        # Remover o pipeline de NER existente, se houver, para evitar conflitos
        if 'ner' in nlp.pipe_names:
            print("Removing existing NER pipeline from the model.")
            nlp.remove_pipe('ner')

        # Criar um novo pipeline de NER e adicionar ao modelo
        print("Adding new NER pipeline to the model.")
        ner = nlp.add_pipe('ner', last=True)

        # Adicionar os rótulos ao componente NER
        print("Adding labels to the NER pipeline.")
        for _, annotations in training_data:
            for ent in annotations.get('entities', []):
                if len(ent) == 3:
                    label = ent[2]
                    if label not in ner.labels:
                        ner.add_label(label)

        # Desativar outros componentes do pipeline durante o treinamento
        other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
        with nlp.disable_pipes(*other_pipes):
            print("Starting training with NER component.")
            try:
                optimizer = nlp.begin_training()
            except Exception as e:
                print(f"Error initializing optimizer: {e}")
                return

            all_losses = []
            for itn in range(n_iter):
                print(f"Starting iteration {itn + 1}.")
                random.shuffle(training_data)
                losses = {}
                batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
                for batch_idx, batch in enumerate(batches):
                    print(f"Processing batch {batch_idx + 1}.")
                    examples = []
                    for text, annotations in batch:
                        print(f"Processing text (first 50 chars): {text[:50]}...")

                        # Remover entidades sobrepostas e garantir que não haja duplicatas
                        annotations['entities'] = remove_overlapping_entities(annotations.get('entities', []))

                        # Garantir que todas as entidades tenham rótulos válidos
                        valid_entities = []
                        for start, end, label in annotations['entities']:
                            if label in ner.labels:
                                valid_entities.append((start, end, label))
                            else:
                                print(f"Warning: Entity label '{label}' not found in NER labels. Skipping this entity.")
                        annotations['entities'] = valid_entities

                        # Criar um exemplo de SpaCy para treinamento
                        try:
                            doc = nlp.make_doc(text)
                            example = Example.from_dict(doc, annotations)
                            examples.append(example)
                        except Exception as e:
                            print(f"Error creating training example for text: {text[:50]}, error: {e}")

                    # Atualizar o modelo com os exemplos de treinamento
                    try:
                        nlp.update(examples, drop=0.2, losses=losses, sgd=optimizer)
                        print(f"Batch {batch_idx + 1} processed, current loss: {losses.get('ner', 0.0)}")
                    except Exception as e:
                        print(f"Error updating model during batch {batch_idx + 1}: {e}")
                        continue

                all_losses.append(losses.get('ner', 0.0))
                print(f"Iteration {itn + 1} completed, Loss: {losses.get('ner', 0.0)}")
    except Exception as e:
        print(f"An error occurred during training: {e}")

    # Plotar a perda de treinamento
    print("Plotting training loss.")
    plot_training_loss(all_losses)

    # Salvar o modelo treinado
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    nlp.to_disk(output_dir)
    print(f"Model saved to {output_dir}")

In [102]:
# Evaluate the model on test data
def evaluate_model(nlp, test_data):
    print("Evaluating the model on test data.")
    y_true = []
    y_pred = []

    for text, annotations in test_data:
        doc = nlp(text)
        true_entities = [(start, end, label) for start, end, label in annotations['entities']]
        pred_entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]

        true_labels = [label for _, _, label in true_entities]
        pred_labels = [ent[2] for ent in pred_entities if (ent[0], ent[1]) in [(start, end) for start, end, _ in true_entities]]

        y_true.extend(true_labels)
        y_pred.extend(pred_labels)

    # Ensure the length of y_true and y_pred match by adding 'None' for missing predictions
    while len(y_true) > len(y_pred):
        y_pred.append('None')

    # Print classification report and accuracy
    print("Classification Report:")
    print(classification_report(y_true, y_pred, zero_division=0))
    print("Accuracy:", accuracy_score(y_true, y_pred))

In [94]:
# Plot training loss (example visualization)
def plot_training_loss(losses):
    plt.plot(losses)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.show()

In [105]:
# Example usage
training_data_path = "C:\\Users\\josed\\codes\\spacy_NER_trainig\\maped_data.json"
output_model_dir = 'C:\\Users\\josed\\codes\\spacy_NER_trainig\\model'

In [106]:
# Load training data
print("Loading training data.")
training_data = load_training_data(training_data_path)

Loading training data.
Loading training data from C:\Users\josed\codes\spacy_NER_trainig\maped_data.json


In [107]:
# Split data into training and testing sets (80% train, 20% test)
print("Splitting data into training and testing sets.")
split_idx = int(0.8 * len(training_data))
train_data = training_data[:split_idx]
test_data = training_data[split_idx:]

Splitting data into training and testing sets.


In [112]:
# Train the model
print("Training the model.")
train_spacy_model(train_data, output_model_dir, n_iter=40)

Training the model.
Loading pre-trained SpaCy model 'pt_core_news_lg'
Model loaded successfully.
Removing existing NER pipeline from the model.
Adding new NER pipeline to the model.
Adding labels to the NER pipeline.
Starting training with NER component.
Error initializing optimizer: [E955] Can't find table(s) lexeme_norm for language 'pt' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.


In [ ]:
# Load the trained model and evaluate it
print("Loading the trained model for evaluation.")
nlp_trained = spacy.load(output_model_dir)
evaluate_model(nlp_trained, test_data)

In [113]:
import spacy
print(spacy.__file__)


C:\Users\josed\codes\script_smm\venv\Lib\site-packages\spacy\__init__.py
